In [1]:
import os
import sys
import time
import pickle
import string
from datetime import datetime
from datetime import timedelta
from itertools import chain, cycle
import collections
from pprint import pprint
from io import StringIO
import pathlib
import sqlite3

import numpy as np
import pandas as pd
import dask.dataframe as dd
from pandas.tools.plotting import parallel_coordinates
import matplotlib.pyplot as plt

#import urllib
#import webbrowser
#import codecs
from bs4 import BeautifulSoup
from xml.dom import minidom
from xml.etree import ElementTree as ET
#from lxml import etree as ET #Supports xpath syntax
import xmltodict 

import nltk
import re
import Levenshtein
import networkx as nx
import json

#from nominatim import Nominatim
#from mpl_toolkits.basemap import Basemap   #http://matplotlib.org/basemap/api/basemap_api.html
from matplotlib import cm
import geopy.geocoders as gg
from geopy.geocoders import Nominatim
import geopandas as gpd
import ipyleaflet
#from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, WMSLayer, Marker, MarkerCluster

##https://python-visualization.github.io/folium/docs-v0.6.0/
#https://towardsdatascience.com/data-101s-spatial-visualizations-and-analysis-in-python-with-folium-39730da2adf
import folium
import folium.plugins
#from folium.plugins import MarkerCluster
#from folium.plugins import FastMarkerCluster


#import plotly as py
#import plotly.graph_objs as go
#import plotly.figure_factory as ff

from IPython.display import Image, HTML, IFrame, FileLink, FileLinks #needed to render in notebook
from IPython.core.display import display

#import pydotplus #Install http://www.graphviz.org/ & Instal https://pypi.python.org/pypi/pydotplus

#https://stackoverflow.com/questions/2148119/how-to-convert-an-xml-string-to-a-dictionary-in-python
#https://github.com/martinblech/xmltodict
#https://www.datacamp.com/community/tutorials/python-xml-elementtree
#https://stackabuse.com/reading-and-writing-xml-files-in-python/
#https://codereview.stackexchange.com/questions/117810/parsing-of-xml-in-python-using-minidom-and-inserting-data-into-a-database
#https://stackoverflow.com/questions/749796/pretty-printing-xml-in-python
#https://hackersandslackers.com/dealing-with-xml-in-python/

In [2]:
df_lemmas = pd.read_pickle('./df_lemmas.pkl')

In [5]:
df_lemmas.columns

Index(['Bundesland', 'Bundesland_idno', 'Gemeinde', 'Gemeinde_idno',
       'Großregion', 'Großregion_idno', 'Kleinregion', 'Kleinregion_idno',
       'Ort', 'Ort_idno', 'archiv', 'bibl', 'certainty', 'cit_def',
       'cit_interp', 'cit_note', 'cit_pRef', 'cit_quote', 'cit_re', 'cit_ref',
       'cit_type', 'date', 'entry_id', 'etym', 'filename', 'fragebogenNummer',
       'hauptlemma_normalized', 'hauptlemma_orig', 'hauptlemma_unique',
       'nebenlemma_normalized', 'nebenlemma_orig', 'nebenlemma_unique',
       'note_anmerkung', 'note_diverse', 'note_notabene', 'orig_placeName',
       'paragraph', 'placeName_id', 'pos', 'quelle', 'quelleBearbeitet',
       'seite', 'simple_placeName', 'tustep', 'usg_corresp', 'verweise'],
      dtype='object')

In [3]:
df_lemmas.head()

,Bundesland,Bundesland_idno,Gemeinde,Gemeinde_idno,Großregion,Großregion_idno,Kleinregion,Kleinregion_idno,Ort,Ort_idno,...,paragraph,placeName_id,pos,quelle,quelleBearbeitet,seite,simple_placeName,tustep,usg_corresp,verweise
0,None,None,None,None,None,None,None,None,None,None,...,None,None,Subst,Egger Gl.,"{1B,1C} hist. TirWeistGl.",30,None,[diaun],None,None
1,None,None,None,None,None,None,None,None,None,None,...,None,None,Subst,Schneider Tir.(1965),"{1B,1C:X} Etym.^@ SCHNEIDER· (1963)",601,Chur,[diaun],None,None
2,None,None,None,None,None,None,None,None,None,None,...,None,None,Subst,Tir.Wb.,"{1B,1C} Tir. TirWb.(1955-1956)","1,1293,337, f337^#3.4 = f.",Chur,[diaun],None,None
3,None,None,None,None,None,None,None,None,None,None,...,None,None,Interj,Straßburg Umg. Nagelmayer,"{2.2d,2.3b,2.3c,2.3d} Gurkt.:MKä.:Kä. FbB.NAG...",None,None,[tib],None,None
4,None,None,None,None,None,None,None,None,None,None,...,None,None,Interj,Straßburg Umg. Nagelmayer,"{2.2d,2.3b,2.3c,2.3d} Gurkt.:MKä.:Kä. FbB.NAG...",None,None,[tib],None,None


In [12]:
df_lemmas[df_lemmas['entry_id'] == 'h361_qdb-d1e9985'].cit_def

558671    [Reisig zu Streu zerhacken]
Name: cit_def, dtype: object

In [11]:
df_lemmas.cit_def = df_lemmas.cit_def.apply(lambda x:x if x != [] else None)